In [1]:
import numpy as np
import pandas as pd
import os

from flipside import Flipside
flipside = Flipside("023b83a2-e411-4849-a961-3742c87bc9c9", "https://api-v2.flipsidecrypto.xyz")

We first set the name of the liquidity pool we want to query data from.

In [2]:
pool_name = "USDC-WETH 500 10"

This write the sql code and start query.

In [3]:
sql = """
SELECT pool_name, action, block_number, block_timestamp, 
       liquidity_provider, nf_token_id,
       amount0_adjusted, token0_symbol, amount0_usd,
       amount1_adjusted, token1_symbol, amount1_usd,
       tick_lower, tick_upper
FROM ethereum.uniswapv3.ez_lp_actions
""" +\
" WHERE pool_name = '" + pool_name + "' " +\
"""
ORDER BY block_number DESC
"""

# Since there is a 30mb rate limit for each page, we need to split the data in seperate pages, we will merge them later
# If we are retrieving a dataset with more columns, we might want to set page_size to a lower number
query_result_set = flipside.query(sql, page_number = 1, page_size = 30000)

Here, we run a loop to merge the pages into one full dataset.

In [4]:
current_page_number = 1
page_size = 30000
# We first set total_pages to 2 (any number larger than current_page_number will do) for the loop to run, we will retrieve the actual total pages in the loop
total_pages = 2

# All the data will be stored in a list called "all_rows"
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

# Merge all rows to a dataframe
df = pd.DataFrame(all_rows)
df.drop(columns = ['__row_index'], inplace = True)
df.head(10)

,pool_name,action,block_number,block_timestamp,liquidity_provider,nf_token_id,amount0_adjusted,token0_symbol,amount0_usd,amount1_adjusted,token1_symbol,amount1_usd,tick_lower,tick_upper
0,USDC-WETH 500 10,INCREASE_LIQUIDITY,17621959,2023-07-04T17:12:35.000Z,0x9aab3f81604c683a1a0d14019fbfe15bef7aa1ee,NaN,4.027749e+05,USDC,NaN,7737.055238,WETH,NaN,200530,200540
1,USDC-WETH 500 10,DECREASE_LIQUIDITY,17621959,2023-07-04T17:12:35.000Z,0x9aab3f81604c683a1a0d14019fbfe15bef7aa1ee,NaN,4.842101e+05,USDC,NaN,7695.396531,WETH,NaN,200530,200540
2,USDC-WETH 500 10,DECREASE_LIQUIDITY,17621878,2023-07-04T16:56:11.000Z,0x1c8f6a5f009e051cab9c3851ca2da2c936b2775a,NaN,9.529047e+06,USDC,NaN,8771.145380,WETH,NaN,200530,200540
3,USDC-WETH 500 10,INCREASE_LIQUIDITY,17621878,2023-07-04T16:56:11.000Z,0x1c8f6a5f009e051cab9c3851ca2da2c936b2775a,NaN,9.577837e+06,USDC,NaN,8746.194670,WETH,NaN,200530,200540
4,USDC-WETH 500 10,DECREASE_LIQUIDITY,17621861,2023-07-04T16:52:47.000Z,0xff82bf5238637b7e5e345888bab9cd99f5ebe331,NaN,1.859375e+07,USDC,NaN,7000.391493,WETH,NaN,200540,200550
5,USDC-WETH 500 10,INCREASE_LIQUIDITY,17621861,2023-07-04T16:52:47.000Z,0xff82bf5238637b7e5e345888bab9cd99f5ebe331,NaN,1.907370e+07,USDC,NaN,6754.758286,WETH,NaN,200540,200550
6,USDC-WETH 500 10,INCREASE_LIQUIDITY,17621857,2023-07-04T16:51:59.000Z,0x042523db4f3effc33d2742022b2490258494f8b3,NaN,1.223680e+07,USDC,NaN,8285.081166,WETH,NaN,200530,200540
7,USDC-WETH 500 10,DECREASE_LIQUIDITY,17621857,2023-07-04T16:51:59.000Z,0x042523db4f3effc33d2742022b2490258494f8b3,NaN,1.204976e+07,USDC,NaN,8380.724704,WETH,NaN,200530,200540
8,USDC-WETH 500 10,INCREASE_LIQUIDITY,17621823,2023-07-04T16:45:11.000Z,0x0cac3d1a887206e0f6169222c4504301a8b4b993,NaN,0.000000e+00,USDC,NaN,8893.357373,WETH,NaN,200490,200500
9,USDC-WETH 500 10,DECREASE_LIQUIDITY,17621823,2023-07-04T16:45:11.000Z,0x0cac3d1a887206e0f6169222c4504301a8b4b993,NaN,7.008945e+04,USDC,NaN,8857.644829,WETH,NaN,200490,200500


Lastly, we want to save the data in our desire repository.

In [15]:
df.to_csv("/Users/zhicong/Dropbox/DEXs/uniswapv3_lp_actions/" + pool_name + ".csv", index=False)